In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

import pandas as pd

# Reading the csv file
df = pd.read_csv('/content/drive/MyDrive/RansomwareData.csv')


Mounted at /content/drive/


In [ ]:
X = df.drop(df.columns[0], axis=1)
X = X.drop(df.columns[1], axis=1)
X = X.drop(df.columns[2], axis=1)
y = df[df.columns[2]]

y_binary = df[df.columns[1]]
# Create group labels
def convert_to_group(label):
    if label in [2, 3, 8, 10]:  # Traditional File Encryptors
        return 1
    elif label in [6, 9]:  # Locker Ransomware
        return 2
    elif label in [5, 7]:  # Ransomware with Trojan-like Behavior
        return 3
    elif label == 1:  # Hybrid and Multifaceted Ransomware
        return 4
    elif label in [4, 11]:  # Specialty Ransomware
        return 5
    else:
        return 0  # Assuming 0 is for goodware or an undefined class

# Apply the grouping function to the multi-class labels
y_group = y.apply(convert_to_group)

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Assume df is your DataFrame and contains your data
X = df.drop([df.columns[0], df.columns[1], df.columns[2]], axis=1).values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit K-means with the desired number of clusters
kmeans = KMeans(n_clusters=4, random_state=42)
cluster_labels = kmeans.fit_predict(X_scaled)

# One-hot encode cluster labels if desired
cluster_labels_one_hot = tf.keras.utils.to_categorical(cluster_labels)

# Concatenate the cluster labels with the original features
X_w_clusters = np.hstack((X, cluster_labels_one_hot))

# Update the input shape
input_shape_with_clusters = X_w_clusters.shape[1]

from sklearn.model_selection import train_test_split
X_with_clusters, X_test_with_clusters, y_train, y_test, y_train_binary, y_test_binary, y_train_group, y_test_group = train_test_split(X_w_clusters, y, y_binary, y_group, test_size=0.2, random_state=42)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, MultiHeadAttention, LayerNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

# Define the TransformerBlock class
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs, training=training)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1, training=training)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
# Input layer
input_layer = Input(shape=(X_with_clusters.shape[1], 1))

# Shared layers
x = Conv1D(filters=256, kernel_size=2, activation='relu')(input_layer)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(filters=128, kernel_size=2, activation='relu')(x)
x = MaxPooling1D(pool_size=2)(x)
x = TransformerBlock(embed_dim=128, num_heads=4, ff_dim=256)(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dense(100, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# Binary Classification Head (Goodware vs Malicious)
binary_output = Dense(1, activation='sigmoid', name='binary_output')(x)

# Group Classification Head
group_output = Dense(6, activation='softmax', name='group_output')(x)

# Specific Classification Head
specific_output = Dense(12, activation='softmax', name='specific_output')(x)

# Create the model
model = Model(inputs=input_layer, outputs=[binary_output, group_output, specific_output])

from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy
from tensorflow.keras import backend as K

# Custom precision metric for multi-class classification
def custom_precision(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

# Custom recall metric for multi-class classification
def custom_recall(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

# Custom F1 score metric for multi-class classification
def custom_f1(y_true, y_pred):
    precision = custom_precision(y_true, y_pred)
    recall = custom_recall(y_true, y_pred)
    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1

model.compile(optimizer='adam',
              loss={'binary_output': 'binary_crossentropy', 'specific_output': 'categorical_crossentropy'},
              metrics={'binary_output': ['accuracy', Precision(), Recall(), custom_f1],
                  'specific_output': [CategoricalAccuracy(), custom_precision, custom_recall, custom_f1]})
# Summary of the model
model.summary()

# Define the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Mapping original labels to group labels
# Assuming original labels are in the range [0, 11]
label_to_group = {
    0: 0,  # Goodware
    1: 4,  # Critroni
    2: 1,  # CryptLocker
    3: 1,  # CryptoWall
    4: 5,  # KOLLAH
    5: 3,  # Kovter
    6: 2,  # Locker
    7: 3,  # MATSNU
    8: 1,  # PGPCODER
    9: 2,  # Reveton
    10: 1,  # TeslaCrypt
    11: 5,  # Trojan-Ransom
}

y_train_group = y_train.map(label_to_group)
y_test_group = y_test.map(label_to_group)

# One-hot encode the group and specific labels
y_train_group_categorical = to_categorical(y_train_group, num_classes=6)
y_train_categorical = to_categorical(y_train, num_classes=12)
y_test_group_categorical = to_categorical(y_test_group, num_classes=6)
y_test_categorical = to_categorical(y_test, num_classes=12)

# Train the model
history = model.fit(X_with_clusters,
                    {'binary_output': y_train_binary, 'group_output': y_train_group_categorical, 'specific_output': y_train_categorical},
                    epochs=15,
                    batch_size=4,
                    validation_data=(X_test_with_clusters, {'binary_output': y_test_binary, 'group_output': y_test_group_categorical, 'specific_output': y_test_categorical}),
                    callbacks=[early_stopping])


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 30971, 1)       │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d (Conv1D)           │ (None, 30970, 256)     │            768 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d             │ (None, 15485, 256)     │              0 │ conv1d[0][0]           │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_1 (Conv1D)         │ (None, 15484, 128)     │         65,664 │ max_pooling1d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_1           │ (None, 7742, 128)      │              0 │ conv1d_1[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_block         │ (None, 7742, 128)      │        330,240 │ max_pooling1d_1[0][0]  │
│ (TransformerBlock)        │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 990976)         │              0 │ transformer_block[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 256)            │    253,690,112 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 100)            │         25,700 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 64)             │          6,464 │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ binary_output (Dense)     │ (None, 1)              │             65 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ group_output (Dense)      │ (None, 6)              │            390 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ specific_output (Dense)   │ (None, 12)             │            780 │ dense_4[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 254,120,183 (969.39 MB)

 Trainable params: 254,120,183 (969.39 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


305/305 ━━━━━━━━━━━━━━━━━━━━ 3425s 11s/step - binary_output_accuracy: 0.5568 - binary_output_custom_f1: 0.2501 - binary_output_precision: 0.4252 - binary_output_recall: 0.4365 - loss: 291.0558 - specific_output_categorical_accuracy: 0.4020 - specific_output_custom_f1: 0.3920 - specific_output_custom_precision: 0.3925 - specific_output_custom_recall: 0.3917 - val_binary_output_accuracy: 0.3836 - val_binary_output_custom_f1: 0.4985 - val_binary_output_precision: 0.3836 - val_binary_output_recall: 1.0000 - val_loss: 15.1374 - val_specific_output_categorical_accuracy: 0.0361 - val_specific_output_custom_f1: 0.0357 - val_specific_output_custom_precision: 0.0357 - val_specific_output_custom_recall: 0.0357
Epoch 2/15
305/305 ━━━━━━━━━━━━━━━━━━━━ 3448s 11s/step - binary_output_accuracy: 0.7357 - binary_output_custom_f1: 0.4333 - binary_output_precision: 0.6543 - binary_output_recall: 0.6119 - loss: 5.3607 - specific_output_categorical_accuracy: 0.5364 - specific_output_custom_f1: 0.5441 - spec

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Generate predictions
predictions = model.predict(X_test_with_clusters)

# Binary predictions and metrics
binary_predictions = (predictions[0] > 0.5).astype(int)  # Convert probabilities to 0 or 1
binary_accuracy = accuracy_score(y_test_binary, binary_predictions)
binary_precision = precision_score(y_test_binary, binary_predictions)
binary_recall = recall_score(y_test_binary, binary_predictions)
binary_f1 = f1_score(y_test_binary, binary_predictions)

# Specific predictions and metrics
specific_predictions = np.argmax(predictions[2], axis=1)
specific_accuracy = accuracy_score(y_test, specific_predictions)
specific_precision = precision_score(y_test, specific_predictions, average='macro')
specific_recall = recall_score(y_test, specific_predictions, average='macro')
specific_f1 = f1_score(y_test, specific_predictions, average='macro')

# Print the results
print(f"Binary Classification Metrics:\n"
      f"Accuracy: {binary_accuracy:.4f}\n"
      f"Precision: {binary_precision:.4f}\n"
      f"Recall: {binary_recall:.4f}\n"
      f"F1 Score: {binary_f1:.4f}")

print(f"\nSpecific Classification Metrics:\n"
      f"Accuracy: {specific_accuracy:.4f}\n"
      f"Precision: {specific_precision:.4f}\n"
      f"Recall: {specific_recall:.4f}\n"
      f"F1 Score: {specific_f1:.4f}")

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Predict on the test set
predictions = model.predict(X_test_with_clusters)

# Binary classification confusion matrix
binary_preds = np.round(predictions[0]).astype(int)
binary_cm = confusion_matrix(y_test_binary, binary_preds)


# Specific classification confusion matrix
specific_preds = np.argmax(predictions[2], axis=1)
specific_cm = confusion_matrix(y_test, specific_preds)

# Plot confusion matrices
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap=cmap, xticklabels=classes, yticklabels=classes)
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

# Plot Binary classification confusion matrix
plot_confusion_matrix(binary_cm, classes=['Goodware', 'Malicious'], title='Binary Classification Confusion Matrix')

# Plot Specific classification confusion matrix
specific_labels = ['Goodware', 'Critroni', 'CryptLocker', 'CryptoWall', 'KOLLAH', 'Kovter', 'Locker', 'MATSNU', 'PGPCODER', 'Reveton', 'TeslaCrypt', 'Trojan-Ransom']
plot_confusion_matrix(specific_cm, classes=specific_labels, title='Specific Classification Confusion Matrix')


In [ ]:

from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
# Plot the model
plot_model(model, to_file='cnt_model.png', show_shapes=True, show_layer_names=True)


import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Display the model
img = mpimg.imread('cnt_model.png')
plt.figure(figsize=(10, 10))
plt.imshow(img)
plt.axis('off')
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Generate predictions
predictions = model.predict(X_test_with_clusters)

# Binary predictions and metrics
binary_predictions = (predictions[0] > 0.5).astype(int)  # Convert probabilities to 0 or 1
binary_accuracy = accuracy_score(y_test_binary, binary_predictions)
binary_precision = precision_score(y_test_binary, binary_predictions)
binary_recall = recall_score(y_test_binary, binary_predictions)
binary_f1 = f1_score(y_test_binary, binary_predictions)

# Specific predictions and metrics
specific_predictions = np.argmax(predictions[1], axis=1)
specific_accuracy = accuracy_score(y_test, specific_predictions)
specific_precision = precision_score(y_test, specific_predictions, average='macro')
specific_recall = recall_score(y_test, specific_predictions, average='macro')
specific_f1 = f1_score(y_test, specific_predictions, average='macro')

# Print the results
print(f"Binary Classification Metrics:\n"
      f"Accuracy: {binary_accuracy:.4f}\n"
      f"Precision: {binary_precision:.4f}\n"
      f"Recall: {binary_recall:.4f}\n"
      f"F1 Score: {binary_f1:.4f}")

print(f"\nSpecific Classification Metrics:\n"
      f"Accuracy: {specific_accuracy:.4f}\n"
      f"Precision: {specific_precision:.4f}\n"
      f"Recall: {specific_recall:.4f}\n"
      f"F1 Score: {specific_f1:.4f}")
